# Automated Feature Selection Engine for Protein Classification

This notebook implements selection framework for the testing data for protein classification. It integrates statistical evaluation, machine learning-based scoring, group interaction modeling, and interpretability analysis, with the objective of identifying an optimal, biologically meaningful subset of features.

---

## Core Principle

Feature selection is treated as a formal optimization problem. Each feature and descriptor group is evaluated based on its predictive value, redundancy, and synergistic interactions — using statistical criteria, cross-validated model performance, and graph-theoretic reasoning.

---

## Workflow Overview

### 1. Feature Matrix Construction

Features are extracted from the following sources:
- `peptides.py`: Physicochemical and QSAR descriptors (e.g., VHSE, Z-scales, Atchley, Kidera)
- `protlearn`: Sequence-derived descriptors (AAC, CTD, autocorrelations, motif, pseudo-AA, QSO)
- `testing_data_w_features.csv`: Dipeptide frequencies, reduced alphabets, basic properties

Features are grouped into logical biological/chemical sets:
- Examples: `"VHSE"`, `"Z-scales"`, `"Dipeptides"`, `"CTD"`, `"Binary Profile"`, `"ProtFP"`

---

In [1]:
# === Core Libraries ===
import numpy as np
import pandas as pd
import re
import json
import itertools
import warnings
import random

# === Scikit-learn: Preprocessing, Models, Feature Selection, CV ===
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import (
    SelectKBest, mutual_info_classif, f_classif, RFE, VarianceThreshold
)
from sklearn.model_selection import (
    StratifiedKFold, cross_val_score, GridSearchCV
)
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.inspection import permutation_importance


# === Bio Feature Engineering ===
import peptides
from peptides import Peptide
from protlearn.features import (
    aac, aaindex1, ngram, entropy, motif, atc, binary,
    cksaap, ctd, ctdc, ctdt, ctdd, moreau_broto, moran,
    geary, paac, apaac, socn, qso
)

# === Advanced Feature Selection ===
from skrebate import ReliefF
from category_encoders import TargetEncoder

# === Optimization, Interpretability, Dimensionality ===
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA
import shap


# === Visualization ===
import matplotlib.pyplot as plt
import seaborn as sns

# === Progress Tracking ===
from tqdm import tqdm
from tqdm.auto import tqdm as tqdm_auto
tqdm.pandas()

# === Disable Warnings ===
warnings.filterwarnings("ignore")


## Step 1: Extraction of Precomputed Features from Input Table

In this step, the precomputed features contained within the `metadata_org_w_features.csv` file were extracted and organized into biologically meaningful components. These features serve as foundational inputs for downstream descriptor generation and classification tasks.

The extracted components include:

- `Entry`, `ProteinClass`: Identifiers and class labels for each protein sequence  
- `Selected_PDB`, `CleanSequence`: Structural metadata and cleaned amino acid sequences  
- `SequenceLength`: The total number of residues present in each protein  
- `freq`: 20-dimensional amino acid composition vectors  
- `dipep`: 400-dimensional dipeptide frequency features capturing sequential residue pairs  
- `red_freq`: Composition profiles based on a reduced amino acid alphabet  
- `red_ngram`: N-gram patterns derived from reduced alphabets  
- `prop`: Physicochemical properties derived using Biopython, including molecular weight, isoelectric point, instability index, and secondary structure propensities

These components were parsed into modular dataframes and preserved for integration with additional feature extraction tools in subsequent steps. This structured approach enables alignment between raw data and advanced descriptor pipelines.

In [3]:
# === Step 1: Load and Parse Evaluation Metadata Table ===

# Load evaluation CSV
data = pd.read_csv("../testing_data_w_features.csv")  # or adjust path as needed

# Verify required columns
assert "CleanSequence" in data.columns, "Missing column: 'CleanSequence'"

# Preview structure
print(f"Dataset shape: {data.shape}")
print("First few columns:", list(data.columns[:10]))

# (DO NOT try to print Label distribution, evaluation set has no labels)

# Metadata extraction
metadata_cols = ["Entry", "Selected_PDB", "CleanSequence", "SequenceLength"]
metadata_df = data[metadata_cols].copy()

# === Step 1.1: Slice and Prefix Basic Feature Blocks ===

# Define slicing schema
i_start = 5  # After metadata columns

# Amino acid composition
freq = data.iloc[:, i_start : i_start + 20]
freq.columns = [f"base_{col}" for col in freq.columns]  # fixed to 'base_'

# Dipeptide frequencies
i_start += 20
dipep = data.iloc[:, i_start : i_start + 400]
dipep.columns = [f"base_{col}" for col in dipep.columns]  # fixed to 'base_'

# Reduced alphabet frequency
i_start += 400
red_freq = data.iloc[:, i_start : i_start + 5]
red_freq.columns = [f"base_{col}" for col in red_freq.columns]  # fixed to 'base_'

# Reduced alphabet N-grams
i_start += 5
red_ngram = data.iloc[:, i_start : i_start + 150]
red_ngram.columns = [f"base_{col}" for col in red_ngram.columns]  # fixed to 'base_'

# Physicochemical properties (Biopython features)
i_start += 150
prop = data.iloc[:, i_start:]
prop.columns = [f"base_{col}" for col in prop.columns]  # fixed to 'base_'

# === Step 1.2: Sanity Checks ===

assert freq.shape[1] == 20, "Expected 20 AAC features"
assert dipep.shape[1] == 400, "Expected 400 dipeptide features"
assert red_freq.shape[1] == 5, "Expected 5 reduced alphabet frequency features"
assert red_ngram.shape[1] == 150, "Expected 150 reduced alphabet ngram features"
assert not prop.isnull().any().any(), "Missing values detected in property descriptors"

# === Step 1.3: Preview ===

print("\nBase AAC feature shape:", freq.shape)
print("Base property feature sample:\n", prop.head(3))


Dataset shape: (171, 588)
First few columns: ['Entry', 'CleanSequence', 'Selected_PDB', 'SequenceLength', 'A', 'C', 'D', 'E', 'F', 'G']

Base AAC feature shape: (171, 20)
Base property feature sample:
    base_IsoelectricPoint  base_Aromaticity  base_InstabilityIndex  \
0               7.865610          0.059590              33.680642   
1               5.223413          0.049521              28.408163   
2               5.655275          0.090411              36.957014   

   base_Flexibility  base_Helix  base_Sheet  base_Turn  base_Gravy  
0          0.999017    0.323091    0.292365   0.384544    0.028026  
1          1.001449    0.263578    0.332268   0.349840   -0.173163  
2          1.000302    0.293151    0.265753   0.345205   -0.498630  


## Step 2: Generation of Peptide Descriptors (`peptides.py`)

In this step, an extended set of **physicochemical and QSAR-inspired descriptors** was generated for each protein sequence using the `peptides.py` library. The `Peptide.descriptors()` method was employed to compute a **102-dimensional feature vector** that numerically summarizes key chemical and structural characteristics of each input sequence.

The descriptors span the following categories:

- **Amino Acid Factor Scores**  
  - VHSE: Principal components derived from hydrophobic, electronic, and steric properties  
  - Atchley Factors: Principal components summarizing amino acid physicochemical attributes  
  - Kidera Factors: Orthogonal scores reflecting structural and physicochemical diversity

- **Substitution Matrix Projections**  
  - BLOSUM1–10: Numerical encodings of evolutionary similarity and substitution patterns

- **Physicochemical Fingerprints and Indices**  
  - Includes ProtFP, PCP, Z-Scales, ST-Scales, MS-WHIM scores  
  - Sneath vectors were also computed, summarizing amino acid similarities in functional space

- **Global Biochemical Properties**  
  - Hydrophobicity, polarity, charge, spatial configuration, and other scalar indices

---

### Naming Convention

All descriptors were systematically **prefixed with `Pep_`** to ensure clear modularity and traceability during downstream analysis. For example: `Pep_VHSE1`, `Pep_Kidera3`, and `Pep_BLOSUM7`. This convention facilitates descriptor group filtering, block-wise feature selection, and structured interpretability analyses.

---

### Rationale

These descriptors provide **global, alignment-independent representations** of peptide sequences and are particularly valuable for capturing overarching chemical signatures. When integrated with localized sequence patterns and structural features (e.g., those computed in Step 3), this hybrid representation enables **robust, interpretable classification models** that generalize across protein families and functional classes.

In [4]:
from peptides import Peptide
from tqdm.auto import tqdm

# === Generate peptide descriptors for each sequence ===

def compute_peptides_descriptors(seq: str) -> dict:
    """
    Generate a dictionary of peptide descriptors for a given protein sequence.

    Parameters:
    - seq (str): A string representation of a clean protein sequence.

    Returns:
    - dict: A dictionary containing 102 physicochemical and QSAR-based descriptors
            as computed by peptides.Peptide.descriptors().
    """
    p = Peptide(seq)
    return p.descriptors()

# Apply the descriptor function to each sequence
peptide_features_df = data['CleanSequence'].progress_apply(compute_peptides_descriptors)

# Convert list of dictionaries into a structured DataFrame
peptide_features_df = pd.DataFrame(peptide_features_df.tolist())

# Fill any NaNs arising from invalid characters
peptide_features_df = peptide_features_df.fillna(0)

# Prefix all columns with 'PEP_'
peptide_features_df.columns = [f"PEP_{col}" for col in peptide_features_df.columns]

# Insert 'Entry' column for clean future merging
peptide_features_df.insert(0, "Entry", data["Entry"].values)

# Save one folder up
peptide_features_df.to_csv("../peptide_eval_descriptors.csv", index=False)

# Preview
print("Peptide descriptor evaluation matrix shape:", peptide_features_df.shape)
print("Example descriptors:", peptide_features_df.columns[:10].tolist())


100%|██████████| 171/171 [00:01<00:00, 95.63it/s] 


Peptide descriptor evaluation matrix shape: (171, 103)
Example descriptors: ['Entry', 'PEP_AF1', 'PEP_AF2', 'PEP_AF3', 'PEP_AF4', 'PEP_AF5', 'PEP_BLOSUM1', 'PEP_BLOSUM2', 'PEP_BLOSUM3', 'PEP_BLOSUM4']


## Step 3: ProtLearn Feature Extraction

In this step, a comprehensive suite of protein sequence descriptors was computed using the `protlearn` library. These descriptors were derived from physicochemical attributes, sequence order information, motif structure, autocorrelation functions, and compositional profiles.

The extracted ProtLearn features were grouped into descriptor families, including:

- **Amino Acid Composition (AAC):** Frequency of each amino acid observed in the sequence.  
- **AAIndex1 Properties:** Averaged values across 553 numeric physicochemical indices.  
- **Dipeptide N-Grams:** Frequencies of adjacent amino acid pairings.  
- **Shannon Entropy:** An information-theoretic measure of intra-sequence variability.  
- **Motif Presence:** Binary indicators of user-defined motifs (e.g., `AAx[KC]`).  
- **ATC Descriptors:** Atomic and bond composition counts for C, H, N, O, S atoms.  
- **Binary Profile:** One-hot encoded identity vectors for residue positions.  
- **CKSAAP:** Composition of amino acid pairs separated by k residues (k=1 by default).  
- **CTD Family:**
  - CTD: Conjoint triad-based sequence groupings
  - CTDC: Composition features
  - CTDT: Transition metrics
  - CTDD: Distribution profiles  
- **Autocorrelations:**
  - Moreau-Broto autocorrelation
  - Moran's I spatial autocorrelation
  - Geary’s C sequence dispersion  
- **Pseudo Amino Acid Compositions (PseAAC):**
  - PAAC: Encodes sequence order with physicochemical attributes
  - APAAC: Adds amphiphilic hydrophobic/hydrophilic bias to PAAC  
- **Sequence Order Descriptors:**
  - SOCN: Sequence-order-coupling numbers (Schneider-Wrede and Grantham distances)
  - QSO: Quasi-sequence-order descriptors using pairwise residue distances

All features were concatenated into a unified `protlearn_df` matrix and integrated into the master dataset. Column names were prefixed with `PL_` to facilitate downstream identification and modular group tracking during block-wise optimization.

This stage enabled rich encoding of structural, chemical, and spatial sequence properties without requiring homology alignment.

In [5]:
# === Step 3: ProtLearn Feature Extraction for Evaluation ===

from protlearn.features import (
    aaindex1, entropy, motif, atc, cksaap, ctd, ctdc, ctdt, ctdd,
    moreau_broto, moran, geary, paac, apaac, socn, qso
)

# Define ProtLearn feature computation function
def compute_protlearn_features(seqs: list[str]) -> dict[str, pd.DataFrame]:
    """
    Computes a suite of ProtLearn-derived protein sequence descriptors.

    The extracted features cover:
        - Physicochemical properties (e.g., AAIndex1, entropy, autocorrelation)
        - Motif and compositional patterns (e.g., CTD, ATC)
        - Order-dependent encodings (e.g., SOCN, QSO)
        - Pseudo amino acid compositions (PAAC, APAAC)

    Parameters:
        seqs (list[str]): A list of amino acid sequences.

    Returns:
        dict[str, pd.DataFrame]: A dictionary of feature blocks, each with its descriptor name as key.
    """
    blocks = {}

    # Physicochemical property summaries
    blocks["AAIndex1"], _ = aaindex1(seqs)

    # Information-theoretic sequence entropy
    blocks["Entropy"] = entropy(seqs)

    # Motif pattern presence
    blocks["Motif"] = motif(seqs, pattern="AAx[KC]")

    # Atom and bond composition
    blocks["ATC_atoms"], blocks["ATC_bonds"] = atc(seqs)

    # K-spaced amino acid pairs
    blocks["CKSAAP"], _ = cksaap(seqs, k=1)

    # CTD (composition, transition, distribution)
    blocks["CTD"], _ = ctd(seqs)
    blocks["CTDC"], _ = ctdc(seqs)
    blocks["CTDT"], _ = ctdt(seqs)
    blocks["CTDD"], _ = ctdd(seqs)

    # Spatial autocorrelations
    blocks["MoreauBroto"] = moreau_broto(seqs)
    blocks["Moran"] = moran(seqs)
    blocks["Geary"] = geary(seqs)

    # Pseudo amino acid compositions
    blocks["PAAC"], _ = paac(seqs, lambda_=3)
    blocks["APAAC"], _ = apaac(seqs, lambda_=3)

    # Order-coupling descriptors
    blocks["SOCN_SW"], blocks["SOCN_Grantham"] = socn(seqs, d=3)
    blocks["QSO_SW"], blocks["QSO_Grantham"], _ = qso(seqs, d=3)

    return blocks

# Extract sequences from the evaluation dataset
seqs_eval: list[str] = data["CleanSequence"].tolist()

# Apply ProtLearn feature extraction
protlearn_blocks_eval = compute_protlearn_features(seqs_eval)

# Format blocks with descriptor-specific prefixes
named_blocks_eval = []
for block_name, block_df in protlearn_blocks_eval.items():
    df_block = pd.DataFrame(block_df)
    df_block.columns = [f"PL_{block_name}_{col}" for col in df_block.columns]
    named_blocks_eval.append(df_block)

# Concatenate all formatted blocks into a unified DataFrame
protlearn_eval_df = pd.concat(named_blocks_eval, axis=1)

# Insert Entry column for future clean merging
protlearn_eval_df.insert(0, "Entry", data["Entry"].values)

# Save ProtLearn features for evaluation one folder up
protlearn_eval_df.to_csv("../protlearn_eval_features.csv", index=False)

# Preview confirmation
print(f"ProtLearn evaluation feature shape: {protlearn_eval_df.shape}")
print("Example ProtLearn features:", protlearn_eval_df.columns[:10].tolist())


ProtLearn evaluation feature shape: (171, 1705)
Example ProtLearn features: ['Entry', 'PL_AAIndex1_0', 'PL_AAIndex1_1', 'PL_AAIndex1_2', 'PL_AAIndex1_3', 'PL_AAIndex1_4', 'PL_AAIndex1_5', 'PL_AAIndex1_6', 'PL_AAIndex1_7', 'PL_AAIndex1_8']
